# Calculate the inventory aging / slow moving items for given Dataset

In [1]:
# # Import different libraries

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import datetime as dt

# Import the inventory dataset

In [2]:
df = pd.read_csv('Inventory management.csv')
df.head()

,Serial_number,Delivery_date,Delivery_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name
0,1,2022-06-22,2022-06,042785AEA04E4909,SKU-042785AEA04E4909,8869,Computing,290,Packed,Germany
1,2,2022-02-17,2022-02,22F2C64EAE484ABF,SKU-22F2C64EAE484ABF,579,Fashion,283,Packed,Spain
2,3,2022-04-28,2022-04,DFFE97ACE1FD4511,SKU-DFFE97ACE1FD4511,7475,Furnitures,263,Packed,France
3,4,2022-07-12,2022-07,98E31AF97D2D44A3,SKU-98E31AF97D2D44A3,1710,Fashion,127,Delivered,France
4,5,2022-11-20,2022-11,A2E255EC93AA4051,SKU-A2E255EC93AA4051,6777,Furnitures,282,Delivered,England


In [3]:
# Chaneg the date columns format

df.Delivery_date= pd.to_datetime(df.Delivery_date)
df.Delivery_month= pd.to_datetime(df.Delivery_month)

# Show the total inventory value

In [4]:
Total_inventory = df.pivot_table(index='Item_Status', values='COGS', columns='Category', aggfunc='sum')
Total_inventory

Category,Computing,Electronics,Fashion,Furnitures,Phones
Item_Status,,,,,
Delivered,31977298,31888947,32070291,32013644,31752290
Packed,8023426,7872841,7927616,7963497,7978080
Received,23993321,23942048,23890145,23887355,23985773
Shipped,7992005,8076762,7896802,7946469,7970491
lost,799715,809335,786869,831052,811180
picked,7273674,7145160,7103062,7186699,7191281


# Get the total loss that needs to be written-off

In [27]:
Lost_items = df[df.Item_Status.isin(['lost'])].copy()
Lost_items.pivot_table(index='Country_Name', columns='Category', values='COGS', aggfunc='sum').round(2)

Category,Computing,Electronics,Fashion,Furnitures,Phones
Country_Name,,,,,
England,117493,120942,121655,122485,124298
France,76848,80901,70163,80686,86664
Germany,159925,167226,159045,174639,155322
Holland,136979,127621,130272,132982,142034
Italy,105891,105397,106209,104166,103783
Poland,38107,43514,41220,41345,33339
Portugal,46514,40648,38667,46181,41954
Spain,117958,123086,119638,128568,123786


# Get the ending inventory balance filtering by item status

In [28]:
inventory_balance = df[df['Item_Status'].isin(['Packed', 'Received', 'picked'])].copy()
inventory_balance.head()

,Serial_number,Delivery_date,Delivery_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name
0,1,2022-06-22,2022-06-01,042785AEA04E4909,SKU-042785AEA04E4909,8869,Computing,290,Packed,Germany
1,2,2022-02-17,2022-02-01,22F2C64EAE484ABF,SKU-22F2C64EAE484ABF,579,Fashion,283,Packed,Spain
2,3,2022-04-28,2022-04-01,DFFE97ACE1FD4511,SKU-DFFE97ACE1FD4511,7475,Furnitures,263,Packed,France
5,6,2022-09-09,2022-09-01,C54EA56FB0354620,SKU-C54EA56FB0354620,3005,Electronics,226,Received,Italy
6,7,2022-07-15,2022-07-01,6F36B76D7F344AE7,SKU-6F36B76D7F344AE7,9751,Computing,271,Received,England


## Calculate the aging per days for the ending inventory balance as of 2022-12-31

In [29]:
inventory_balance['Aging']= inventory_balance['Delivery_date'].apply(lambda x: (dt.datetime(2022,12,31) - x).days)
inventory_balance.head()

,Serial_number,Delivery_date,Delivery_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name,Aging
0,1,2022-06-22,2022-06-01,042785AEA04E4909,SKU-042785AEA04E4909,8869,Computing,290,Packed,Germany,192
1,2,2022-02-17,2022-02-01,22F2C64EAE484ABF,SKU-22F2C64EAE484ABF,579,Fashion,283,Packed,Spain,317
2,3,2022-04-28,2022-04-01,DFFE97ACE1FD4511,SKU-DFFE97ACE1FD4511,7475,Furnitures,263,Packed,France,247
5,6,2022-09-09,2022-09-01,C54EA56FB0354620,SKU-C54EA56FB0354620,3005,Electronics,226,Received,Italy,113
6,7,2022-07-15,2022-07-01,6F36B76D7F344AE7,SKU-6F36B76D7F344AE7,9751,Computing,271,Received,England,169


# Calculate the provision for the inventory assuming the company policy as below:

## .less than 90 days           = Zero provision
## .From 91 : 120 days        = 30% provision
## .From 121 : 180 days      = 60% provision
## .Over 180 days                = 100% provision


In [30]:
def a(inventory_balance):
    if inventory_balance['Aging'] <= 90:
        val = 0
    elif inventory_balance['Aging'] > 90 and inventory_balance['Aging'] <= 120:
        val = inventory_balance['COGS'] * .3
    elif inventory_balance['Aging'] > 120 and inventory_balance['Aging'] <= 180:
        val = inventory_balance['COGS'] * .6
    else:
        val = inventory_balance['COGS'] * 1
    return val

inventory_balance['Provision'] = inventory_balance.apply(a, axis=1)
inventory_balance.set_index('Serial_number').head()

,Delivery_date,Delivery_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name,Aging,Provision
Serial_number,,,,,,,,,,,
1,2022-06-22,2022-06-01,042785AEA04E4909,SKU-042785AEA04E4909,8869,Computing,290,Packed,Germany,192,290.0
2,2022-02-17,2022-02-01,22F2C64EAE484ABF,SKU-22F2C64EAE484ABF,579,Fashion,283,Packed,Spain,317,283.0
3,2022-04-28,2022-04-01,DFFE97ACE1FD4511,SKU-DFFE97ACE1FD4511,7475,Furnitures,263,Packed,France,247,263.0
6,2022-09-09,2022-09-01,C54EA56FB0354620,SKU-C54EA56FB0354620,3005,Electronics,226,Received,Italy,113,67.8
7,2022-07-15,2022-07-01,6F36B76D7F344AE7,SKU-6F36B76D7F344AE7,9751,Computing,271,Received,England,169,162.6


# Arrange the data

In [31]:
Provision = inventory_balance[['Delivery_date','Delivery_month','UID','SKU', 'Category','COGS'
                  ,'Item_Status','Country_Name', 'Aging', 'Provision']]
Provision.head()

,Delivery_date,Delivery_month,UID,SKU,Category,COGS,Item_Status,Country_Name,Aging,Provision
0,2022-06-22,2022-06-01,042785AEA04E4909,SKU-042785AEA04E4909,Computing,290,Packed,Germany,192,290.0
1,2022-02-17,2022-02-01,22F2C64EAE484ABF,SKU-22F2C64EAE484ABF,Fashion,283,Packed,Spain,317,283.0
2,2022-04-28,2022-04-01,DFFE97ACE1FD4511,SKU-DFFE97ACE1FD4511,Furnitures,263,Packed,France,247,263.0
5,2022-09-09,2022-09-01,C54EA56FB0354620,SKU-C54EA56FB0354620,Electronics,226,Received,Italy,113,67.8
6,2022-07-15,2022-07-01,6F36B76D7F344AE7,SKU-6F36B76D7F344AE7,Computing,271,Received,England,169,162.6


# Show the provision for each Country per Category

In [32]:
country_provision = inventory_balance.pivot_table(index=['Country_Name'], columns=['Category'], values=['Provision'],aggfunc='sum')
country_provision

Provision                                             
Category      Computing Electronics    Fashion Furnitures     Phones
Country_Name                                                        
England       3724950.0   3654065.2  3699167.5  3666202.2  3682156.3
France        2458106.1   2455768.3  2440310.1  2458549.4  2484100.6
Germany       4897410.6   4889797.8  4920268.2  4872184.4  4889542.3
Holland       4185003.6   4215194.2  4140885.1  4166657.2  4131672.5
Italy         3146870.4   3114099.6  3213813.9  3183950.9  3195891.2
Poland        1240511.4   1243247.4  1182005.8  1232302.8  1244788.7
Portugal      1240147.8   1205175.3  1212340.9  1224735.1  1217042.2
Spain         3731119.3   3681576.4  3645461.6  3697611.0  3699809.5